In [1]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas as pd
from multiprocessing import Pool
from vtk.util import numpy_support
from sklearn.decomposition import PCA
from numpy import linalg as LA
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import pickle
from sklearn import manifold
from numpy import linalg as LA

In [2]:
def read_multiblock(filename):
    reader = vtk.vtkXMLMultiBlockDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()
    
def write_multiblock(filename,data):
    writer = vtk.vtkXMLMultiBlockDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()

def ensureUtf(s):
    try:
        if type(s) == unicode:
            return s.encode('utf8', 'ignore')
    except: 
        return str(s)
    
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def read_vtp(filename):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
def write_vtp(filename,data):
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()
    
def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    return x + dimx*(y+dimy*z)

def extract_target_feature(feature_volume,confidence_th,feature_id):
    
    feature_volume.GetPointData().SetActiveScalars('feature_similarity')
    thresholding = vtk.vtkThreshold()
    thresholding.ThresholdByUpper(confidence_th)
    thresholding.SetInputData(feature_volume)
    seg = vtk.vtkConnectivityFilter()
    seg.SetInputConnection(thresholding.GetOutputPort())
    seg.SetExtractionModeToAllRegions()
    seg.ColorRegionsOn()
    thresholding2 = vtk.vtkThreshold()
    thresholding2.SetInputConnection(seg.GetOutputPort())
    thresholding2.ThresholdBetween(feature_id,feature_id)
    thresholding2.Update()
    return thresholding2.GetOutput()

def compute_1d_indices(single_feature,dims,gbounds):
    
    oneD_indices = [] 
    
    for i in range(single_feature.GetNumberOfPoints()):
        pts = single_feature.GetPoint(i)
        xval = int(((pts[0] - gbounds[0])/(gbounds[1]-gbounds[0]))*dims[0])
        yval = int(((pts[1] - gbounds[2])/(gbounds[3]-gbounds[2]))*dims[1])
        zval = int(((pts[2] - gbounds[4])/(gbounds[5]-gbounds[4]))*dims[2])
        
        if xval > dims[0]-1:
            xval = dims[0]-1
        
        if yval > dims[1]-1:
            yval = dims[1]-1
            
        if zval > dims[2]-1:
            zval = dims[2]-1    
        
        index = compute_3d_to_1d_map(xval,yval,zval,dims[0],dims[1],dims[2])
        oneD_indices.append(index)
        
        if index > dims[0]*dims[1]*dims[2] or index < 0:
            print (xval,yval,zval)
    
    if len(oneD_indices) != single_feature.GetNumberOfPoints():
        print ('mismatch in mumber of points!!')
        
    return oneD_indices    

In [3]:
## Parameters
confidence_th = 0.92
size_threshold = 100
dims  = [128,16,128]
initT = 75
endT = 408

feature_tstep = 100 #(in Paraview should subtract 75)

feature_path = '../out/mfix_local_grid/'
feature_fname = feature_path + 'slic_compare_' + str(feature_tstep) + '.vti'
particle_data_path = '../out/isolated_particles/'
particle_out_path = '../out/segmented_particles/'
data_path = '../out/segmented_features/'
feature_volume = read_vti(feature_fname)
gbounds = feature_volume.GetBounds()

In [4]:
feature_volume.GetPointData().SetActiveScalars('feature_similarity')
thresholding = vtk.vtkThreshold()
thresholding.ThresholdByUpper( confidence_th )
thresholding.SetInputData(feature_volume)
segmentation = vtk.vtkConnectivityFilter()
segmentation.SetInputConnection(thresholding.GetOutputPort())
segmentation.SetExtractionModeToAllRegions()
segmentation.ColorRegionsOn()
segmentation.Update()
ug = segmentation.GetOutput()

## load filtered particle data
particle_fname = particle_data_path + '/filtered_particles_' + str(feature_tstep) + '.vtp'
pdata = read_vtp(particle_fname)

target_particle_1d_pts = compute_1d_indices(pdata,dims,gbounds)

#load pickle file
pickle_fname = data_path + 'feature_values_' + str(feature_tstep) + '.pickle'
ffname = ensureUtf(pickle_fname)
fname = open(ffname, 'rb')
data = pickle.load(fname, encoding='latin1')
data = np.asarray(data)
filtered_feature_ids = data[:,1]

## create the time step specific output folder
mode = 0o777
out_folder = particle_out_path + '/segmented_particles_' + str(feature_tstep)
os.mkdir(out_folder, mode)

for i in filtered_feature_ids:    
    thresholding2 = vtk.vtkThreshold()
    thresholding2.SetInputData(ug)
    thresholding2.ThresholdBetween(i,i)
    thresholding2.Update()
    
    target_feature_1d_pts = compute_1d_indices(thresholding2.GetOutput(),dims,gbounds)
    
    intersection_indices = set.intersection(*[set(target_feature_1d_pts), set(target_particle_1d_pts)])
    
    xy, x_ind, y_ind = np.intersect1d(numpy.array(list(intersection_indices)),
                                      np.asarray(target_particle_1d_pts), return_indices=True)
    
    ## Now isolate the particles by matched indices
    filtered_pdata = vtk.vtkPolyData()
    filtered_vtp_points = vtk.vtkPoints()
    filtered_vtp_points.SetNumberOfPoints(len(y_ind))
    
    ## Isolating using y_ind indices since those indices correspond to indices in target_particle_1d_pts
    idx=0
    for j in y_ind:
        filtered_vtp_points.InsertPoint(idx,pdata.GetPoint(j))
        idx=idx+1
    
    filtered_pdata.SetPoints(filtered_vtp_points)
    
    ## write the vtp file out
    filtered_pdata_fname = out_folder + '/feature_' + str(feature_tstep) + '_' + str(i) + '.vtp'
    write_vtp(filtered_pdata_fname,filtered_pdata)